In [ ]:
# Manipulación y procesamiento de Datos
# ==============================================================================
import pandas as pd
import numpy as np

# Plot - Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf


# Definir el tamano del grafico
# ==============================================================================
from pylab import rcParams
rcParams['figure.figsize'] = (18,7)

# No presentar advertencia
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")

# Cargar los datos

Los datos

se requiere installar

`pip install openpyxl`

In [ ]:
data = pd.read_excel("./siniestro_info_mercado_ecuador.xlsx")
data

In [ ]:
# Dummy COVID (mar–jun 2020)
data["covid_dummy"] = 0
data.loc[(data["fecha"] >= np.datetime64("2020-04")) & (data["fecha"] <= np.datetime64("2020-12")), "covid_dummy"] = 1


In [ ]:
data["covid_dummy"].unique()

In [ ]:
data.info()

In [ ]:
data.describe(include= ["float"])

In [ ]:
data.columns

In [ ]:
dark_style = {
    'axes.facecolor': "#AB915E", # '#484366'  '#008080' "#abc9ea","#98daa7" ,"#f3aba8"  ,"#d3c3f7", "#f3f3af",  "#c0ebe9"
    'axes.grid': True,}  
plt.rcParams.update(dark_style)

In [ ]:
plt.figure(figsize=(18, 6))
sns.lineplot(x='fecha', y = 'tasa_siniestralidad', data = data, )
plt.title('Tasa de Siniestralidad')
#plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(18, 6))
sns.lineplot(x='fecha', y = 'prima_nrd', data = data)
plt.title('Prima Neta Devengada')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(18, 6))
sns.lineplot(x='fecha', y = 'costo_siniestros', data = data)
plt.title('costo_siniestros')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Setting up the seaborn style
sns.set(style="whitegrid")

# Histogram for the 'price' variable
plt.figure(figsize=(12, 7))
sns.histplot(data["tasa_siniestralidad"], kde=True, color='skyblue', bins=25, alpha=0.7, line_kws={'linewidth': 2, 'color': 'blue'})
plt.title('Tasa de Siniestralidad', fontsize=16)
#plt.xlabel('Price', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Adding mean and median lines
mean_price = data["tasa_siniestralidad"].mean()
median_price = data["tasa_siniestralidad"].median()
plt.axvline(mean_price, color='red', linestyle='--', linewidth=2, label=f'Mean: ${mean_price:.2f}')
plt.axvline(median_price, color='green', linestyle='-', linewidth=2, label=f'Median: ${median_price:.2f}')

# Adding a legend
plt.legend()
plt.grid(False)

# Displaying the plot
plt.show()

In [ ]:
data['Day'] = data['fecha'].dt.day_name().astype('category')
data['Month'] = data['fecha'].dt.month_name().astype('category')

In [ ]:
day_ordered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data['Day'] = pd.Categorical(data['Day'], categories=day_ordered, ordered=True)

month_ordered = ['January', 'February', 'March', 'April', 'May', 'June', 
                 'July', 'August', 'September', 'October', 'November', 'December']

data['Month'] = pd.Categorical(data['Month'], categories=month_ordered, ordered=True)

In [ ]:
# Set the figure and axes
fig, ax = plt.subplots(figsize=(18,6))

# Plot the sales trend across the date
sns.lineplot(x = 'Month', y = 'tasa_siniestralidad', data = data, ci=None, marker='o', color='darkblue', ax=ax)

# Despine the right side
sns.despine(right=True)

# Set the label
ax.set_xlabel('Month', color='#6F7378')
ax.set_ylabel('Frecuencia de Siniestralidad', color='#6F7478')

# Change the spine color
for spine in ax.spines.values():
    spine.set_edgecolor('#6F7378')

# Set the title
ax.set_title('Tasa de Siniestralidad - Mensual', weight='bold', pad=30, size=14, x=-0.115,
             color='#6F7378', ha='left') 

# Change tick color
ax.tick_params(color='#6F7378')

plt.show()

In [ ]:
# Boxplot graph for monthly seasonality
# ==============================================================================
df1 = data.copy()
plt.style.use('classic')
fig, ax = plt.subplots(figsize=(18, 7))
df1['month'] = df1['fecha'].apply(lambda x: x.month)
df1.boxplot(column = 'tasa_siniestralidad', by='month', ax=ax, color="red")
df1.groupby('month')["tasa_siniestralidad"].median().plot(style='o-', linewidth=0.9, ax=ax)
ax.set_ylabel('Beer')
ax.set_title('Estacionalidad mensual: Tasa de Siniestralidad')
fig.suptitle('');
#plt.savefig("Gráfico de Barra-mes")

In [ ]:
# Boxplot graph for annual seasonality
# ==============================================================================
# Extract year component from date
df1['year'] = data['fecha'].dt.year

# Create a dictionary to store the data for each year
data_anual = {}
for year in df1['year'].unique():
    data_anual[year] = df1.loc[df1['year'] == year, 'tasa_siniestralidad'].values

# Create a data list for the Boxplot chart
boxplot_data = [data_anual[year] for year in sorted(data_anual.keys())]

# Create the Boxplot chart
plt.figure(figsize=(18, 6))
plt.boxplot(boxplot_data, labels=sorted(data_anual.keys()))
plt.xlabel('year')
plt.ylabel('tasa_siniestralidad')
plt.title('Estacionalidad Anual: tasa_siniestralidad')

# Add the median line graph
medians = [np.median(data_anual[year]) for year in sorted(data_anual.keys())]
plt.plot(range(1, len(medians) + 1), medians, marker='o', color='red', linestyle='-')

plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize = (18, 6))

# Cambiar fondo de cada subplot
for ax in axs.flat:
    ax.set_facecolor("#C4A0A9")  # color amarillo claro
    ax.grid(True, linestyle='--', alpha=0.6)  # grilla activada
    ax.set_axisbelow(True)                   # Grilla detrás de los datos

plot_acf(data["tasa_siniestralidad"],  lags = 60, ax = axs[0],color="fuchsia")
axs[0].set_title("Autocorrelation");

# Grafico
plot_pacf(data["tasa_siniestralidad"],  lags = 60, ax=axs[1],color="lime")
axs[1].set_title('Partial Autocorrelation')

#plt.savefig("Gráfico de Densidad y qq")
plt.show();

# Analisis de correlacion

La correlacion ayuda a seleccionar variable

- Definir que tipo de correlacion quiero
    - Exista correlacion entre las variables (entre mas alto mejor)
    - No Correlacion (entre mas bajo mejor)
    - Exista correlacion inversa(valor negativo mas alto mejor)

In [ ]:
plt.figure(figsize= (18,6))
sns.heatmap(data.corr(numeric_only= True),annot=True, cmap="coolwarm")
plt.title("Correlación")
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose 
add = seasonal_decompose(data["tasa_siniestralidad"], model = "add", period = 12)
add.plot();

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt

# Prueba Dickey Fuller
from statsmodels.tsa.stattools import adfuller
from numpy import log
def Augmented_Dickey_Fuller_Test_func(series , column_name):
    print (f'Resultados de la prueba de Dickey-Fuller para columna: {column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','No Lags Used','Número de observaciones utilizadas'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05: # P - Values
        print("Conclusion:====>")
        print("Rechazar la hipótesis nula")
        print("Los datos son estacionarios")
    else:
        print("Conclusion:====>")
        print("No se puede rechazar la hipótesis nula")
        print("Los datos no son estacionarios")

In [ ]:
Augmented_Dickey_Fuller_Test_func(data["tasa_siniestralidad"],"Tasa de Siniestralidad")

In [ ]:
Augmented_Dickey_Fuller_Test_func(data["tasa_siniestralidad"].diff().dropna(),"Tasa de Siniestralidad")

In [ ]:
Augmented_Dickey_Fuller_Test_func(data["tasa_siniestralidad"].diff().diff().dropna(),"Tasa de Siniestralidad")

In [ ]:
def tsplot(y, lags=None, figsize=(18, 7), style='bmh'): # [3]
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Gráfico de analisys de Serie de Tiempo\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
tsplot(data["tasa_siniestralidad"].diff().diff().dropna(), lags = 40);

Necesitamos conocer los terminos de los modelo AR y del modelo MA:

- Para el modelo AR, necesitamos visualizar el grafico PACF.

- Para el modelo MA, necesitamos visualizar el grafico de ACF.

## `GridSearch del Modelo ARIMA`

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings("ignore")

# Supongamos que tienes una serie temporal llamada 'y'
# y = tu_serie_temporal

# Rango de valores para p y q
p_range = range(0, 5)
q_range = range(0, 5)

# Guardar resultados
resultados = []

for p in p_range:
    for q in q_range:
        if p == 0 and q == 0:
            continue  # ARMA(0,0) no tiene sentido
        try:
            modelo = ARIMA(data["tasa_siniestralidad"], order=(p, 2, q))
            resultado = modelo.fit()
            resultados.append({
                'p': p,
                'i': 2,
                'q': q,
                'AIC': resultado.aic,
                'BIC': resultado.bic
            })
        except:
            continue

# Convertir a DataFrame
df_resultados = pd.DataFrame(resultados)

In [ ]:
# Ordenar por AIC
print("Top modelos por AIC:")
print(df_resultados.sort_values('AIC').head())

# Ordenar por BIC
print("\nTop modelos por BIC:")
print(df_resultados.sort_values('BIC').head())

ARIMA(p, i, q)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

pivot = df_resultados.pivot(index='p', columns='q', values='AIC')
plt.figure(figsize=(18, 6))
sns.heatmap(pivot, annot=True, fmt=".1f", cmap="YlGnBu")
plt.title("Mapa de calor de AIC para combinaciones (p, q)")
plt.xlabel("q")
plt.ylabel("p")
plt.tight_layout()
plt.show()

## Division de los Datos 

Dividamos nuestros datos en conjuntos.
1. Datos para entrenar nuestro modelo
2. Datos para probar nuestro modelo.

Para los datos de prueba utilizaremos los últimos 12 meses para probar y evaluar el rendimiento de nuestro modelo.

In [ ]:
sini = data[['fecha', 'costo_siniestros', 'prima_nrd', 'tasa_siniestralidad','ipc_transporte', 'precipitacion']]
sini = sini.set_index('fecha')
sini

In [ ]:
train = sini[sini.index<= '2024-07-01'] 
test = sini[sini.index>'2024-07-01'] 

train.shape, test.shape

###  se requiere instalar 
`!pip install pmdarima`

Para instalar `pmdarima` necesitas crear un entorno nuevo con una version `Numpy` inferior

In [ ]:

from pmdarima import auto_arima

In [ ]:
# Ajustar modelo ARIMA no estacional
modelo = auto_arima(
    train["tasa_siniestralidad"],
    seasonal=False,     # 🔑 Desactiva la estacionalidad
    stepwise=True,      # Búsqueda eficiente
    suppress_warnings=True,
    trace=True           # Muestra el proceso de búsqueda
)

## Modelo ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit the ARIMA model

model_ARIMA = ARIMA(train["tasa_siniestralidad"],order = (1,1,0),)

# Entrenar el modelo
arima_fit = model_ARIMA.fit()

# Mostrar resumen del Model
print(arima_fit.summary())

In [ ]:
import scipy.stats as stats

residuales =pd.DataFrame(arima_fit.resid.iloc[1:], columns = ["residuo"])

fig, axs = plt.subplots(nrows=2, ncols=2)

# plot[1,1]
residuales["residuo"].plot(ax=axs[0,0])
axs[0,0].set_title("Residuals");
axs[0,0].axhline(0, linestyle='--', color='gray')

# plot
sns.distplot(residuales["residuo"], ax=axs[0,1]);
axs[0,1].set_title("Density plot - Residual");

# plot
stats.probplot(residuales["residuo"], dist="norm", plot=axs[1,0])
axs[1,0].set_title('Plot Q-Q')

# plot
plot_acf(residuales["residuo"],  lags=35, ax=axs[1,1],color="fuchsia")
axs[1,1].set_title("Autocorrelation");

plt.show();

In [ ]:
test

In [ ]:
# forecast = arima.forecast(steps = 5)
forecast = arima_fit.forecast(steps = 12)
forecast

In [ ]:
# plot forcasting
plt.figure(figsize=(18,6))
plt.plot(sini["tasa_siniestralidad"], label='Actual')
#plt.plot(weekly_venta_log[-90:], label='Actual')
plt.plot(forecast, label='Forecast')
plt.title('MA Model Forecast')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

#  RMSE
arma_rmse1 = np.sqrt(mean_squared_error(test['tasa_siniestralidad'], forecast))

print(f'Modelo ARIMA RMSE: {arma_rmse1}')

## `Modelo Sarima`

In [ ]:
from pmdarima import auto_arima

# Ajustar modelo ARIMA estacional
modelo = auto_arima(
    train["tasa_siniestralidad"],
    seasonal = True,        # 🔑 Activar estacionalidad
    m = 12,                 # Periodo estacional (12 para mensual, 7 para semanal, etc.)
    stepwise=True,
    trace=True,
    suppress_warnings=True
)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarima_model= SARIMAX(train['tasa_siniestralidad'], order = (1,1,0), seasonal_order = (2,0,0,12), )
sarima_fit = sarima_model.fit()
print(sarima_fit.summary())

In [ ]:
sarima_forecast = sarima_fit.forecast(steps = 12)
sarima_forecast

In [ ]:
# plot forcast
plt.figure(figsize=(18,6))
plt.plot(sini["tasa_siniestralidad"], label='Actual')
plt.plot(sarima_forecast, label='SARIMA Forecast')
plt.title('SARIMA Forecast')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

#  RMSE
arima_rmse2 = np.sqrt(mean_squared_error(test['tasa_siniestralidad'], sarima_forecast))

print(f'Modelo MA RMSE: {arima_rmse2}')

## `Modelo SARIMAX`

In [ ]:
data.columns

In [ ]:
sini2 = data[['fecha', 'costo_siniestros', 'prima_nrd', 'tasa_siniestralidad','ipc_transporte', 'precipitacion', 'riesgo_pais', 'feriados_dias', 'covid_dummy']]

# Renombrar variables
sini2 = sini2.rename({"fecha": "ds", "tasa_siniestralidad": "y"}, axis = 1)
sini2["unique_id"] = "Tasa de Siniestralidad"
sini2

In [ ]:
sini2 = sini2.set_index('ds')

In [ ]:
sini2.columns

## VIF
El método VIF (Variance Inflation Factor) es una técnica estadística utilizada para detectar multicolinealidad entre variables independientes en un modelo de regresión. En otras palabras, mide cuánto se “inflan” las varianzas de los coeficientes debido a la correlación entre predictores.

###  ¿Qué significa el VIF?

$$
\text{VIF}_i = \frac{1}{1 - R_i^2}
$$

Donde $ R_i^2 $ es el coeficiente de determinación de la regresión de la variable $ i $ contra todas las demás variables independientes.

---

### 📊 Interpretación de los valores

| VIF        | Interpretación 📘                          |
|------------|--------------------------------------------|
| 1          | No hay multicolinealidad                   |
| 1–5        | Multicolinealidad moderada (aceptable)     |
| > 5        | Multicolinealidad alta (precaución)        |
| > 10       | Multicolinealidad severa (revisar modelo)  |


### 🧭 ¿Por qué es útil?

- Te ayuda a **diagnosticar redundancia** entre variables.
- Mejora la **interpretabilidad** del modelo.
- Permite **reducir ruido** y evitar sobreajuste.


### `Interpretación del VIF`

El VIF indica cuánto aumenta la varianza de un coeficiente de regresión debido a la correlación 

con otras variables predictoras. Los umbrales comunes son:

$$\text{VIF}_j = \frac{1}{1 - R_{j}^2} $$

- VIF < 5: Sin multicolinealidad significativa.

- 5 < VIF < 10: Multicolinealidad moderada, tolerable en algunos casos.

- VIF > 10: Multicolinealidad alta, problemática para modelos lineales.


In [ ]:
sini2.columns

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Suponiendo tus datos imputados (data_imputed del código anterior)
reduced_vars = [
 'prima_nrd', 'covid_dummy',
 'precipitacion', 'riesgo_pais', 'feriados_dias'
]
X_reduced = sini2[reduced_vars]

vif_reduced = pd.DataFrame()
vif_reduced['Variable'] = X_reduced.columns
vif_reduced['VIF'] = [variance_inflation_factor(X_reduced.values, i) for i in range(X_reduced.shape[1])]

print("VIF con variables reducidas:")
print(vif_reduced)

#### `Agregar rezagos`

- Muestra los valores pasado de la series

In [ ]:
num_lags = 5
for lag in range(1, num_lags + 1):
    sini2[f'lag{lag}'] = sini2['y'].shift(lag)

#sini2.dropna(inplace = True)


### `Agregar promedios moviles`

- Promedio movil: muestra los erroes de la series temporal

In [ ]:
# Lista de tamaños de ventana
windows = [2, 3, 4]

# Calcular promedios móviles para cada tamaño de ventana
for window in windows:
    sini2[f'moving_average_{window}'] = sini2['y'].rolling(window=window).mean()

sini2.dropna(inplace = True)

# Mostrar el DataFrame con los promedios móviles
sini2

In [ ]:
train2 = sini2[sini2.index<= '2024-07-01'] 
test2 = sini2[sini2.index>'2024-07-01'] 

train2.shape, test2.shape

In [ ]:
train2

- costo siniestro = el gasto del siniestro 

- tasa de siniestralidad = costo_siniestra/ pnrd

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarimax_model = SARIMAX(train2['y'], 
                       exog = train2[['costo_siniestros', 'precipitacion', 'riesgo_pais',  'moving_average_2', 'lag1', 'moving_average_3', 'moving_average_4']], # variable exogena
                        order = (0,1,1),  # parte no estacional
                       seasonal_order = (0,1,0,12)) # SMA(12) con periodo 52



# Entrenar el modelo
sarimax_fit = sarimax_model.fit()

# Mostrar el resumen del Modelo

print(sarimax_fit.summary())

prima_nrd', 'precipitacion', 'lag1', 'moving_average_2',

In [ ]:
# Forecast
#sarmax_pred = sarma_fit.forecast(steps=len(test), exog = test[["checkout_price", 'base_price']])
sarimax_pred = sarimax_fit.predict(start = "2024-08-01", end = "2025-07-01" , 
                                   exog = test2[['costo_siniestros', 'precipitacion', 'riesgo_pais',  'moving_average_2', 'lag1', 'moving_average_3', 'moving_average_4']])
sarimax_pred

In [ ]:
test2[['y']]

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [ ]:
sarimax_rmse = np.sqrt(mean_squared_error(test2['y'], sarimax_pred))
print(f'SARIMAX RMSE: {sarimax_rmse}')

In [ ]:
# Cálculo del MAPE
sarimax_mape = mean_absolute_percentage_error(test2['y'], sarimax_pred) * 100
print(f'SARIMAX MAPE: {sarimax_mape}')

si el error de la industria de la aseguradora de auto sinestrado es 1.79% a 2.0%

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(train2['y'][-100:], label='Actual')
plt.plot(test2['y'], label='test')
plt.plot(sarimax_pred, label='SARIMAX - Forecast')
#plt.plot(sarima_forecast, label='SARIMA Forecast')
plt.title('test VS arima_pred')
plt.grid()
plt.legend()
plt.show()

In [ ]:
pd.DataFrame(sarimax_fit.resid.iloc[1:], columns = ["residuo"])

In [ ]:
import scipy.stats as stats

residuales_smax =pd.DataFrame(sarimax_fit.resid.iloc[1:], columns = ["residuo"])

fig, axs = plt.subplots(nrows=2, ncols=2)

# plot[1,1]
residuales_smax["residuo"].plot(ax=axs[0,0])
axs[0,0].set_title("Residuals");
axs[0,0].axhline(0, linestyle='--', color='gray')

# plot
sns.distplot(residuales_smax["residuo"], ax=axs[0,1]);
axs[0,1].set_title("Density plot - Residual");

# plot
stats.probplot(residuales_smax["residuo"], dist="norm", plot=axs[1,0])
axs[1,0].set_title('Plot Q-Q')

# plot
plot_acf(residuales_smax["residuo"],  lags=35, ax=axs[1,1],color="fuchsia")
axs[1,1].set_title("Autocorrelation");

plt.show();

# Modelo con AutoArima 

Se requiere instalar `Statsforecat`

    - pip install statsforecast

In [ ]:
df = sini2.copy()

df = df.reset_index()
df

In [ ]:
sini3 = df[['ds', 'y','costo_siniestros', 'precipitacion', 'riesgo_pais',  'moving_average_2', 'lag1', 'moving_average_3', 'moving_average_4', 'unique_id']]

sini3

In [ ]:
train3 = sini3[sini3["ds"]<= '2024-07-01'] 
test3 = sini3[sini3["ds"] >'2024-07-01'] 

train3.shape, test3.shape

In [ ]:
# Modelo Clasicos
# ==============================================================================
from statsforecast import StatsForecast
from statsforecast.utils import ConformalIntervals
from utilsforecast.plotting import plot_series
from statsforecast.models import AutoARIMA
from statsforecast.models import SeasonalNaive
from statsforecast.arima import arima_string

In [ ]:
season_length = 12 # diario

# Llamamos al modelo que vamos a usar
models = [AutoARIMA(season_length = season_length), 
          ] # 

In [ ]:
# Construimos el modelo
sf = StatsForecast(
                   models = models,
                   freq = 'MS', 
                   fallback_model = SeasonalNaive(season_length = season_length),
                   n_jobs = -1)

In [ ]:
from statsforecast.utils import ConformalIntervals

# Intervalo de forecasting - Predicciones conforme
intervals = ConformalIntervals(h = 12, n_windows = 5) # Cross Validation 

# fit the models
sf.fit(df = train3, prediction_intervals = intervals)

In [ ]:
arima_string(sf.fitted_[0,0].model_)

In [ ]:
test3.drop("y",axis=1, inplace=True)

In [ ]:
# Prediction
Y_hat1 = sf.forecast(df = train3, h = 12, level = [80, 95], fitted = True, X_df = test3)  #  X_df = test
Y_hat1

In [ ]:
plot_series(sini3, Y_hat1, max_insample_length = 100, level = [80, 95])

In [ ]:
cv_sf = sf.cross_validation(df = train3, h = 12, n_windows = 5)
cv_sf.tail()

In [ ]:
from utilsforecast.losses import mse, mae, rmse, mape, smape

def evaluate_cv(df, metric):
    models = df.columns.drop(['unique_id', 'ds', 'y', 'cutoff']).tolist()
    evals = metric(df, models=models)
    evals['best_model'] = evals[models].idxmin(axis=1)
    return evals

In [ ]:
evaluate_cv(cv_sf, rmse)

In [ ]:
plot_series(sini3, Y_hat1, max_insample_length = 100, level = [80, 95])

## `Modelos de ML`

- Se requiere instalar MLForecast
    - `pip install mlforecast`

### Seleccionar las variable que usaron en el modelo Sarimax:

- costo_siniestros,
- precipitacion
- riesgo_pais  
- lag1
- moving_average_2
- moving_average_3 
- moving_average_4

## Divir datos

- Datos en entrenamiento
- Datos de prueba(o validar el modelo)

Usaremos los ultimos 12 meses para validar o probar los modelos

In [ ]:
train4 = sini3[sini3["ds"]<= '2024-07-01'] 
test4 = sini3[sini3["ds"] >'2024-07-01'] 

train4.shape, test4.shape

### Cargar librera 

In [ ]:
# Utils - Herramientas
# ==============================================================================
from utilsforecast.plotting import plot_series
from utilsforecast.preprocessing import fill_gaps
from functools import partial
from utilsforecast.feature_engineering import fourier, trend, time_features
from utilsforecast.feature_engineering import pipeline
from utilsforecast.feature_engineering import future_exog_to_historic
from mlforecast.target_transforms import LocalRobustScaler, LocalStandardScaler
from mlforecast.lag_transforms import RollingMean, ExpandingStd
import operator
from mlforecast.target_transforms import Differences
from mlforecast.utils import PredictionIntervals
from mlforecast.lag_transforms import Combine

In [ ]:
# Sklearn modelos
# ==============================================================================
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

# Mlforecast
# ==============================================================================
from mlforecast import MLForecast


## Seleccionar los modelo a entrenar

- Regresion lineal
- Arbol de Decison
- Random Forest(bosque aleatorio)
- XGBoost(Boosting)

Luego crear un diccionario

In [ ]:
# Crear diccionario 

model1 = {
    'lr':LinearRegression(),
    "rf": RandomForestRegressor(n_estimators= 500, max_depth= 5, n_jobs= -1),
    'xgb': XGBRegressor(n_estimators = 500,       # Número de árboles (más árboles = mejor ajuste)
                        max_depth = 5,            # Profundidad de los árboles (mayor profundidad captura más patrones)
                        ),
    "dt": DecisionTreeRegressor(),
}

## Construir el modelo

Mandamos a llamar a la funcion `MLForecast`, con los siguientes parametros:

- Models: Modelos que se han seleccionado
- freq: Frecuencia de los datos(Mensuales)
- target_transforms: Transforma la variables (Diferenciar la serie o estandarizar la serie)
- date_feature: Datos estacionales
- num_treads: Número de subprocesos a utilizar al calcular las características.

In [ ]:
mlf1 = MLForecast(models = model1,
                 freq = 'MS', 
                 #lags = range(1, 5, 1),
                 target_transforms= [Differences([12]), LocalStandardScaler()], # LocalRobustScaler(scale='iqr')  Differences([1]), LocalStandardScaler()
                 date_features = ["year", "month", "day"], # Estacionalidad
                 num_threads = 32
                 ) 

In [ ]:
mlf1.preprocess(train, static_features= [])

## `Entrenar los modelos`

In [ ]:
# Entrenar el modelo
mlf1.fit(train4, fitted = True, static_features= [],
prediction_intervals=PredictionIntervals(n_windows = 5, h = 12, method="conformal_distribution"))

## `Forecasting`

- h: El horizonte(lo que quiero pronosticar, e.g= mensuales)
- level: Intervalos de predicciones (nivel de confianza, en percentil)
- X_df: Variables exogenas(datos de entrenamiento)

In [ ]:
forecast1 = mlf1.predict(h = 12, level = [80,95], X_df = test4) 

forecast1.tail()

### Visualziar el forecast

- usamos la funcion `plot_series`

In [ ]:
plot_series(sini3, forecast1,  max_insample_length = 100, )

In [ ]:
# Agregar intervalo de predicciones al 95
plot_series(sini3, forecast1,  max_insample_length = 100, level = [95])

In [ ]:
# Agregar intervalo de predicciones al 80
plot_series(sini3, forecast1,  max_insample_length = 100, level = [80])

In [ ]:
# Agregar intervalo de predicciones al cambinado [80, 95]
plot_series(sini3, forecast1,  max_insample_length = 100, level = [80, 95])

# **Evaluar el rendimiento de los modelo.** 

## **Perform time series cross-validation**

In [ ]:
cv1 = mlf1.cross_validation(
    train4, static_features= [],
    n_windows = 5,  # número de modelos a entrenar/divisiones a realizar
    h = 12,  
)
cv1.tail()

## Evaluar alguna metrica el modelo

- mse
- mae
- rmse
- mape
- smape

In [ ]:
from utilsforecast.losses import mse, mae, rmse, mape, smape

def evaluate_cv(df, metric):
    models = df.columns.drop(['unique_id', 'ds', 'y', 'cutoff']).tolist()
    evals = metric(df, models=models)
    evals['best_model'] = evals[models].idxmin(axis=1)
    return evals

In [ ]:
# evaluar con rmse
evaluate_cv(cv1, rmse)

In [ ]:
# evaluar con mape
evaluate_cv(cv1, mape)

In [ ]:
plot_series(sini3, forecast1,  max_insample_length = 300, models = ["rf"],)

In [ ]:
plot_series(sini3, forecast1,  max_insample_length = 100, models = ["rf"], level = [80, 95])